In [2]:
import pandas as pd
import numpy as np

In [3]:
# Import data
data_original = pd.read_csv('../data/other/raw_signal_data.csv', index_col = 0)
labels_data = pd.read_csv('../data/emotional_data/emotion_labels.csv', index_col = 0)

/Users/joeydekruis/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
# Function for combining the sentences and matching them with the right emotional value
def stackRows():
    # specify dictionary for results
    dic = {"averaged": {}, "arousal": {}, "valence": {}, "features": {}}
    # Specify the number of features we have
    num_cols = len(data_original.loc[:, "Fp1":].columns)
    
    # Loop through respondents
    for resp in np.unique(data_original["respondent"]):
        # Loop through sentences
        for item in np.unique(data_original[data_original["respondent"] == resp][".id"]):
            # Create id that is represented by Id.[respondent].[sentence]. to locate the label
            labelId = [i for i, el in enumerate(labels_data["ids"]) if "Id.{}.{}.".format(resp, item) in str(el)]
            # Check if the labelId is not empty
            if len(labelId) != 0:
                labelId = labelId[0]
                # Specify the sentenceId so we can merge the data based on the sentence
                sentenceId = "Sen.{}.{}.".format(resp, labels_data["item"][labelId])
                # Check if label already exists
                if sentenceId not in dic["features"]:
                    # Add features to dic if it doesn't exist
                    dic["features"][sentenceId] = np.array(data_original[(data_original["respondent"] == resp) & 
                                                (data_original[".id"] == item)].iloc[:,3:]).reshape(1, -1, num_cols)
                    # Add valence and arousal
                    dic["arousal"][sentenceId] = labels_data["arousal"][labelId]
                    dic["valence"][sentenceId] = labels_data["valence"][labelId]
                    dic["averaged"][sentenceId] = 1
                else:
                    # Merge features with already existing features
                    temp_data = np.array(data_original[(data_original["respondent"] == resp) & 
                                        (data_original[".id"] == item)].iloc[:,3:]).reshape(1, -1, num_cols)
                    dic["features"][sentenceId] = np.hstack((dic["features"][sentenceId], temp_data))
                    # Calculate average
                    arousal = [labels_data["arousal"][labelId], dic["arousal"][sentenceId]]
                    dic["arousal"][sentenceId] = np.sum(arousal)
                    valence = [labels_data["valence"][labelId], dic["valence"][sentenceId]]
                    dic["valence"][sentenceId] = np.sum(valence)
                    dic["averaged"][sentenceId] += 1
    delete = []
    # Chech which sentences just have an adjective or just a noun and delete them
    for i in dic["averaged"]:
        if dic["averaged"][i] == 1:
            delete += [i]
        if dic["averaged"][i] == 3:
            delete += [i]
    for i in delete:
        dic["averaged"].pop(i, None)
        dic["arousal"].pop(i, None)
        dic["valence"].pop(i, None)
        dic["features"].pop(i, None)
                
    return dic
            
data = stackRows()

In [18]:
# Get result in the right frame
features = [*data["features"].values()][0]
for i in data["features"].keys():
    if i != list(data["features"].keys())[0]:
        features = np.vstack((features,data["features"][i]))
        
labels = np.vstack((np.array([*data["arousal"].values()]), np.array([*data["valence"].values()])))

In [19]:
np.save("../data/other/features_raw.npy", features)
np.save("../data/other/labels_summed.npy", labels)